In [ ]:
import sys
print("Python executable:", sys.executable)

import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np

import optuna
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (StratifiedKFold, cross_val_score, 
                                     train_test_split)
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import ExtraTreesClassifier

# Modelos
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import pickle
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

# Configuraciones de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.show_dimensions', True)


app_train_def_7 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v7.csv')

In [ ]:
X = app_train_def_7.drop(columns = ['TARGET'])
y = app_train_def_7['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [ ]:
lgb_best_paramas = {
    
}

xgb_best_params = {
    
}



lgb_model_base = LGBMClassifier(**lgb_best_params)
xgb_model_base = XGBClassifier(**xgb_best_params)

lgb_meta_params = {
    "n_estimators": 2000,
    "learning_rate": 0.01,
    "random_state": 42,
    "n_jobs": 20
}
lgb_model_meta = LGBMClassifier(**lgb_meta_params)

In [ ]:
stack_clf = StackingClassifier(
    estimators=[
        ("lgbm", lgb_model_base),
        ("xgb", xgb_model_base),
    ],
    modelo_final=lgb_model_meta,  
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),  
    n_jobs=20,
    passthrough=False  
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(stack_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=20)
print(f"Stacking CV Mean AUC: {scores.mean():.4f}")

In [ ]:
stack_clf.fit(X_train, y_train)

In [ ]:
y_pred_proba = stack_clf.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_proba)
print(f"Stacking AUC on Test: {auc_test:.4f}")